In [1]:
# import the libraries
import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import copy
import os.path as osp

#import torch
from torch.nn import ModuleList
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch.nn import MSELoss

from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.utils import k_hop_subgraph
from torch_geometric.loader import ClusterData, ClusterLoader, NeighborSampler, NeighborLoader

# import the functions
from numpy import dot
import torch.nn as nn
from sklearn import metrics
from numpy.linalg import norm
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

# import the datasets
from torch_geometric.datasets import Planetoid
from tqdm import tqdm

In [2]:
dynamic_data = np.load('data/preprocessed/dynamic_data.npy')
static_data = np.load('data/preprocessed/static_data.npy')

In [5]:
print(dynamic_data)

(2628, 279)


In [6]:
!unzip GTCNN.zip

Archive:  GTCNN.zip
   creating: GTCNN/.ipynb_checkpoints/
  inflating: GTCNN/.ipynb_checkpoints/parametric_pooling_net-checkpoint.py  
   creating: GTCNN/__pycache__/
  inflating: GTCNN/__pycache__/parametric_pooling_net.cpython-310.pyc  
  inflating: GTCNN/__pycache__/parametric_pooling_net_ordering.cpython-310.pyc  
   creating: GTCNN/components/
   creating: GTCNN/components/__pycache__/
  inflating: GTCNN/components/__pycache__/evaluation.cpython-310.pyc  
  inflating: GTCNN/components/__pycache__/GGRNN.cpython-310.pyc  
  inflating: GTCNN/components/__pycache__/graph_utils.cpython-310.pyc  
  inflating: GTCNN/components/__pycache__/lsigf.cpython-310.pyc  
  inflating: GTCNN/components/__pycache__/parametric_graph_filter.cpython-310.pyc  
  inflating: GTCNN/components/__pycache__/plot_utils.cpython-310.pyc  
  inflating: GTCNN/components/__pycache__/pred_utils.cpython-310.pyc  
  inflating: GTCNN/components/__pycache__/space_time_pooling.cpython-310.pyc  
  inflating: GTCNN/compon

In [7]:
split = [.3, .2, .5] # [.3, .2, .5]; train, validation, test portions -- summed up to 1
pred_horizen = 1
obs_window = 4
n_listings = dynamic_data.shape[0]
n_timestamps = dynamic_data.shape[1]

In [8]:
from GTCNN.components.train_utils import create_forecasting_dataset

dataset = create_forecasting_dataset(dynamic_data,
                                     splits = split,
                                     pred_horizen= pred_horizen,
                                     obs_window= obs_window,
                                     in_sample_mean= False)

dataset has been created.
-------------------------
79 train data points
52 validation data points
136 test data points


In [30]:
x_train = dataset['trn']['data']
x_test = dataset['tst']['data']
x_val = dataset['val']['data']
y_val = dataset['val']['labels']
y_train = dataset['trn']['labels']
y_test = dataset['trn']['labels']

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, hidden_size, in_dim,out_dim,in_head = 8,out_head = 1):
        super(GAT, self).__init__()
        self.hid = hidden_size
        self.in_head = in_head
        self.out_head = out_head
        
        
        self.conv1 = GATv2Conv(in_dim, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATv2Conv(self.hid*self.in_head, out_dim, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
                
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)
    

criterion = MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

model = GAT(hidden_size=8, in_dim = x_train.shape[2], out_dim = x_train.shape[1]).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    out = model(x_train)
    loss = criterion(out, y_train)
    
    if epoch%200 == 0:
        print(loss)
    
    loss.backward()
    optimizer.step()
    
    

In [ ]:
model.eval()
_, pred = model(x_test).max(dim=1)
correct = float(pred.eq(y_test).sum().item())
acc = correct / x_test.sum().item()
print('Accuracy: {:.4f}'.format(acc))